<a href="https://colab.research.google.com/github/mrRR7/Rerouting-Model/blob/main/Rerouting_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import random
from collections import defaultdict
import json
from datetime import datetime, timedelta
import math

# Data structures for fleet optimization
@dataclass
class Location:
    lat: float
    lon: float

@dataclass
class Order:
    id: str
    location: Location
    demand: float
    time_window: Tuple[float, float]  # (start_hour, end_hour)

@dataclass
class Vehicle:
    id: str
    capacity: float
    depot: Location
    max_shift_hours: float
    current_busy: bool = False

@dataclass
class Scenario:
    id: str
    vehicles: List[Vehicle]
    orders: List[Order]
    weather_code: int  # 0=clear, 1=rain, 2=fog, 3=storm
    hour_of_day: int
    traffic_index: float  # 0.0 to 1.0

# Weather multipliers for ETA adjustment
WEATHER_MULTIPLIERS = {
    0: 1.0,   # clear
    1: 1.2,   # rain
    2: 1.1,   # fog
    3: 1.3,   # storm
}

class RouteCandidate:
    """Represents a candidate route for a vehicle"""
    def __init__(self, vehicle_id: str, route: List[str], features: Dict):
        self.vehicle_id = vehicle_id
        self.route = route  # List of order IDs
        self.features = features

class FleetDataGenerator:
    """Generates synthetic training data for fleet optimization"""

    def __init__(self, use_real_apis: bool = False):
        self.use_real_apis = use_real_apis

    def calculate_distance(self, loc1: Location, loc2: Location) -> float:
        """Calculate Haversine distance between two locations"""
        R = 6371  # Earth radius in km
        lat1, lon1 = math.radians(loc1.lat), math.radians(loc1.lon)
        lat2, lon2 = math.radians(loc2.lat), math.radians(loc2.lon)

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        return R * c

    def estimate_base_eta(self, loc1: Location, loc2: Location, traffic_index: float) -> float:
        """Estimate base ETA in hours (simulated if not using real APIs)"""
        distance = self.calculate_distance(loc1, loc2)
        base_speed = 40  # km/h average city speed
        traffic_factor = 1 + (traffic_index * 0.5)  # Traffic can slow by up to 50%
        return (distance / base_speed) * traffic_factor

    def generate_candidate_routes(self, scenario: Scenario, vehicle: Vehicle,
                                 k_candidates: int = 5) -> List[RouteCandidate]:
        """Generate K candidate routes for a vehicle using different strategies"""
        candidates = []

        # Get orders that can be served by this vehicle
        available_orders = [o for o in scenario.orders]

        for strategy_idx in range(k_candidates):
            # Different strategies for route generation
            if strategy_idx == 0:
                # Nearest neighbor
                route = self._nearest_neighbor_route(vehicle, available_orders, scenario)
            elif strategy_idx == 1:
                # Random insertion
                route = self._random_insertion_route(vehicle, available_orders)
            elif strategy_idx == 2:
                # Time window priority
                route = self._time_window_priority_route(vehicle, available_orders)
            else:
                # Random shuffles
                route = self._random_shuffle_route(vehicle, available_orders)

            # Calculate features for this route
            features = self._extract_route_features(vehicle, route, scenario)

            candidates.append(RouteCandidate(vehicle.id, route, features))

        # Add OR-Tools optimal route as the last candidate (simulated here)
        optimal_route = self._simulated_ortools_route(vehicle, available_orders, scenario)
        optimal_features = self._extract_route_features(vehicle, optimal_route, scenario)
        candidates.append(RouteCandidate(vehicle.id, optimal_route, optimal_features))

        return candidates

    def _nearest_neighbor_route(self, vehicle: Vehicle, orders: List[Order],
                               scenario: Scenario) -> List[str]:
        """Generate route using nearest neighbor heuristic"""
        route = []
        current_loc = vehicle.depot
        remaining_orders = orders.copy()
        current_capacity = 0

        while remaining_orders and current_capacity < vehicle.capacity:
            # Find nearest unvisited order
            min_dist = float('inf')
            nearest_order = None

            for order in remaining_orders:
                if current_capacity + order.demand <= vehicle.capacity:
                    dist = self.calculate_distance(current_loc, order.location)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_order = order

            if nearest_order:
                route.append(nearest_order.id)
                current_loc = nearest_order.location
                current_capacity += nearest_order.demand
                remaining_orders.remove(nearest_order)
            else:
                break

        return route

    def _random_insertion_route(self, vehicle: Vehicle, orders: List[Order]) -> List[str]:
        """Generate route using random insertion"""
        route = []
        shuffled_orders = orders.copy()
        random.shuffle(shuffled_orders)
        current_capacity = 0

        for order in shuffled_orders:
            if current_capacity + order.demand <= vehicle.capacity:
                route.append(order.id)
                current_capacity += order.demand

        return route

    def _time_window_priority_route(self, vehicle: Vehicle, orders: List[Order]) -> List[str]:
        """Generate route prioritizing time windows"""
        # Sort by time window start
        sorted_orders = sorted(orders, key=lambda o: o.time_window[0])
        route = []
        current_capacity = 0

        for order in sorted_orders:
            if current_capacity + order.demand <= vehicle.capacity:
                route.append(order.id)
                current_capacity += order.demand

        return route

    def _random_shuffle_route(self, vehicle: Vehicle, orders: List[Order]) -> List[str]:
        """Generate random route respecting capacity"""
        shuffled = orders.copy()
        random.shuffle(shuffled)
        route = []
        current_capacity = 0

        for order in shuffled:
            if current_capacity + order.demand <= vehicle.capacity:
                route.append(order.id)
                current_capacity += order.demand

        return route

    def _simulated_ortools_route(self, vehicle: Vehicle, orders: List[Order],
                                scenario: Scenario) -> List[str]:
        """Simulate OR-Tools optimal route (in practice, call actual OR-Tools)"""
        # This simulates what OR-Tools would return - a near-optimal route
        # In production, replace with actual OR-Tools VRP solver
        return self._nearest_neighbor_route(vehicle, orders, scenario)

    def _extract_route_features(self, vehicle: Vehicle, route: List[str],
                               scenario: Scenario) -> Dict:
        """Extract features for a route candidate"""
        if not route:
            return self._empty_features()

        # Create order lookup
        order_map = {o.id: o for o in scenario.orders}

        # Calculate route metrics
        total_base_eta = 0.0
        total_distance = 0.0
        total_demand = 0.0
        num_tw_violations = 0
        sum_lateness = 0.0
        max_single_leg_eta = 0.0

        current_loc = vehicle.depot
        current_time = scenario.hour_of_day

        for order_id in route:
            order = order_map[order_id]

            # Calculate leg metrics
            leg_distance = self.calculate_distance(current_loc, order.location)
            leg_eta = self.estimate_base_eta(current_loc, order.location, scenario.traffic_index)

            total_distance += leg_distance
            total_base_eta += leg_eta
            total_demand += order.demand
            max_single_leg_eta = max(max_single_leg_eta, leg_eta)

            # Check time window violation
            arrival_time = current_time + leg_eta
            if arrival_time > order.time_window[1]:
                num_tw_violations += 1
                sum_lateness += (arrival_time - order.time_window[1]) * 60  # Convert to minutes

            current_loc = order.location
            current_time = arrival_time

        # Add return to depot
        return_eta = self.estimate_base_eta(current_loc, vehicle.depot, scenario.traffic_index)
        total_base_eta += return_eta
        total_distance += self.calculate_distance(current_loc, vehicle.depot)

        # Apply weather multiplier
        weather_mult = WEATHER_MULTIPLIERS[scenario.weather_code]
        total_adjusted_eta = total_base_eta * weather_mult

        # Calculate remaining shift hours
        remaining_shift = max(0, vehicle.max_shift_hours - total_adjusted_eta)

        features = {
            'adj_eta_hours': total_adjusted_eta,
            'base_eta_hours': total_base_eta,
            'distance_km': total_distance,
            'stops_count': len(route),
            'capacity_utilization': total_demand / vehicle.capacity if vehicle.capacity > 0 else 0,
            'num_tw_violations': num_tw_violations,
            'sum_lateness_minutes': sum_lateness,
            'weather_code': scenario.weather_code,
            'hour_of_day_sin': np.sin(2 * np.pi * scenario.hour_of_day / 24),
            'hour_of_day_cos': np.cos(2 * np.pi * scenario.hour_of_day / 24),
            'traffic_index': scenario.traffic_index,
            'vehicle_remaining_shift_hours': remaining_shift,
            'vehicle_busy_flag': 1 if vehicle.current_busy else 0,
            'max_single_leg_eta': max_single_leg_eta,
        }

        return features

    def _empty_features(self) -> Dict:
        """Return features for empty route"""
        return {
            'adj_eta_hours': 0.0,
            'base_eta_hours': 0.0,
            'distance_km': 0.0,
            'stops_count': 0,
            'capacity_utilization': 0.0,
            'num_tw_violations': 0,
            'sum_lateness_minutes': 0.0,
            'weather_code': 0,
            'hour_of_day_sin': 0.0,
            'hour_of_day_cos': 0.0,
            'traffic_index': 0.0,
            'vehicle_remaining_shift_hours': 0.0,
            'vehicle_busy_flag': 0,
            'max_single_leg_eta': 0.0,
        }

    def generate_training_data(self, num_scenarios: int = 1000,
                              k_candidates: int = 5) -> pd.DataFrame:
        """Generate complete training dataset"""
        all_data = []

        for scenario_idx in range(num_scenarios):
            # Generate random scenario
            scenario = self._generate_random_scenario(scenario_idx)

            for vehicle in scenario.vehicles:
                # Generate candidate routes
                candidates = self.generate_candidate_routes(scenario, vehicle, k_candidates)

                # Find best candidate (last one is OR-Tools optimal)
                best_idx = len(candidates) - 1

                # Create training examples
                for idx, candidate in enumerate(candidates):
                    row = {
                        'scenario_id': scenario.id,
                        'vehicle_id': candidate.vehicle_id,
                        'candidate_index': idx,
                        **candidate.features,
                        'label': 1 if idx == best_idx else 0
                    }
                    all_data.append(row)

        return pd.DataFrame(all_data)

    def _generate_random_scenario(self, idx: int) -> Scenario:
        """Generate a random scenario for training"""
        # Random number of vehicles and orders
        num_vehicles = random.randint(2, 5)
        num_orders = random.randint(5, 20)

        # Generate depot location (centered around a city)
        depot_lat = 40.7128 + random.uniform(-0.1, 0.1)  # NYC area
        depot_lon = -74.0060 + random.uniform(-0.1, 0.1)
        depot = Location(depot_lat, depot_lon)

        # Generate vehicles
        vehicles = []
        for v_idx in range(num_vehicles):
            vehicle = Vehicle(
                id=f"v_{idx}_{v_idx}",
                capacity=random.uniform(100, 500),
                depot=depot,
                max_shift_hours=random.uniform(6, 10),
                current_busy=random.random() < 0.3
            )
            vehicles.append(vehicle)

        # Generate orders
        orders = []
        for o_idx in range(num_orders):
            # Orders scattered around depot
            order_lat = depot_lat + random.uniform(-0.2, 0.2)
            order_lon = depot_lon + random.uniform(-0.2, 0.2)

            # Random time windows
            tw_start = random.uniform(6, 18)
            tw_end = tw_start + random.uniform(2, 6)

            order = Order(
                id=f"o_{idx}_{o_idx}",
                location=Location(order_lat, order_lon),
                demand=random.uniform(10, 50),
                time_window=(tw_start, min(tw_end, 24))
            )
            orders.append(order)

        # Random scenario conditions
        scenario = Scenario(
            id=f"scenario_{idx}",
            vehicles=vehicles,
            orders=orders,
            weather_code=random.choice([0, 0, 0, 1, 1, 2, 3]),  # More clear weather
            hour_of_day=random.randint(0, 23),
            traffic_index=random.uniform(0, 1)
        )

        return scenario

class FleetRouteDataset(Dataset):
    """PyTorch dataset for fleet routing"""

    def __init__(self, df: pd.DataFrame):
        self.df = df
        # Group by scenario and vehicle for batch processing
        self.groups = list(df.groupby(['scenario_id', 'vehicle_id']))

    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        (scenario_id, vehicle_id), group_df = self.groups[idx]

        # Feature columns
        feature_cols = [
            'adj_eta_hours', 'base_eta_hours', 'distance_km', 'stops_count',
            'capacity_utilization', 'num_tw_violations', 'sum_lateness_minutes',
            'weather_code', 'hour_of_day_sin', 'hour_of_day_cos', 'traffic_index',
            'vehicle_remaining_shift_hours', 'vehicle_busy_flag', 'max_single_leg_eta'
        ]

        features = torch.FloatTensor(group_df[feature_cols].values)
        labels = torch.LongTensor(group_df['label'].values)

        # Get the index of the best candidate
        best_idx = torch.where(labels == 1)[0][0]

        return features, best_idx

class RouteRankingModel(nn.Module):
    """Small MLP for ranking route candidates"""

    def __init__(self, input_dim: int = 14, hidden_dim: int = 64):
        super(RouteRankingModel, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1)  # Output single score per candidate
        )

    def forward(self, x):
        # x shape: (batch_size, num_candidates, num_features)
        batch_size, num_candidates, num_features = x.shape

        # Reshape for processing
        x_flat = x.view(-1, num_features)

        # Get scores
        scores = self.model(x_flat)

        # Reshape back
        scores = scores.view(batch_size, num_candidates)

        return scores

class FleetOptimizationTrainer:
    """Training and inference for fleet optimization model"""

    def __init__(self, model: RouteRankingModel, device: str = 'cpu'):
        self.model = model.to(device)
        self.device = device

    def train(self, train_loader: DataLoader, val_loader: DataLoader,
              epochs: int = 50, lr: float = 1e-3):
        """Train the route ranking model"""
        optimizer = optim.Adam(self.model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        train_losses = []
        val_accuracies = []

        for epoch in range(epochs):
            # Training phase
            self.model.train()
            epoch_loss = 0.0

            for batch_features, batch_labels in train_loader:
                batch_features = batch_features.to(self.device)
                batch_labels = batch_labels.to(self.device)

                optimizer.zero_grad()

                # Get scores for all candidates
                scores = self.model(batch_features)

                # Calculate loss
                loss = criterion(scores, batch_labels)

                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

            avg_train_loss = epoch_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation phase
            val_acc = self.evaluate(val_loader)
            val_accuracies.append(val_acc)

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f}, Val Acc: {val_acc:.4f}")

        return train_losses, val_accuracies

    def evaluate(self, data_loader: DataLoader) -> float:
        """Evaluate model accuracy"""
        self.model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch_features, batch_labels in data_loader:
                batch_features = batch_features.to(self.device)
                batch_labels = batch_labels.to(self.device)

                scores = self.model(batch_features)
                predictions = torch.argmax(scores, dim=1)

                correct += (predictions == batch_labels).sum().item()
                total += batch_labels.size(0)

        return correct / total if total > 0 else 0.0

    def predict(self, candidates_features: np.ndarray) -> Tuple[int, np.ndarray]:
        """
        Predict best route from candidates

        Args:
            candidates_features: Array of shape (num_candidates, num_features)

        Returns:
            best_idx: Index of best candidate
            scores: Scores for all candidates
        """
        self.model.eval()

        with torch.no_grad():
            features = torch.FloatTensor(candidates_features).unsqueeze(0).to(self.device)
            scores = self.model(features).squeeze(0)
            best_idx = torch.argmax(scores).item()

        return best_idx, scores.cpu().numpy()

class FleetOptimizationPipeline:
    """Complete pipeline for fleet optimization"""

    def __init__(self, model_path: Optional[str] = None):
        self.data_generator = FleetDataGenerator()
        self.model = RouteRankingModel()
        self.trainer = FleetOptimizationTrainer(self.model)

        if model_path:
            self.load_model(model_path)

    def train_model(self, num_scenarios: int = 1000, epochs: int = 50):
        """Train the optimization model"""
        print("Generating training data...")
        df = self.data_generator.generate_training_data(num_scenarios)

        # Split into train/val
        train_size = int(0.8 * len(df.groupby(['scenario_id', 'vehicle_id'])))
        all_groups = list(df.groupby(['scenario_id', 'vehicle_id']))
        random.shuffle(all_groups)

        train_groups = all_groups[:train_size]
        val_groups = all_groups[train_size:]

        train_df = pd.concat([group for _, group in train_groups])
        val_df = pd.concat([group for _, group in val_groups])

        # Create datasets
        train_dataset = FleetRouteDataset(train_df)
        val_dataset = FleetRouteDataset(val_df)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        print("Training model...")
        train_losses, val_accs = self.trainer.train(train_loader, val_loader, epochs)

        print(f"Final validation accuracy: {val_accs[-1]:.4f}")

        return train_losses, val_accs

    def optimize_routes(self, scenario: Scenario, k_candidates: int = 5) -> Dict[str, List[str]]:
        """
        Optimize routes for all vehicles in a scenario

        Returns:
            Dictionary mapping vehicle_id to optimal route (list of order_ids)
        """
        optimal_routes = {}

        for vehicle in scenario.vehicles:
            # Generate candidates
            candidates = self.data_generator.generate_candidate_routes(
                scenario, vehicle, k_candidates
            )

            # Extract features
            features = np.array([list(c.features.values()) for c in candidates])

            # Predict best route
            best_idx, scores = self.trainer.predict(features)

            # Get the best route
            optimal_routes[vehicle.id] = candidates[best_idx].route

            print(f"Vehicle {vehicle.id}: Selected candidate {best_idx} with score {scores[best_idx]:.3f}")
            print(f"  Route: {candidates[best_idx].route[:5]}..." if len(candidates[best_idx].route) > 5 else f"  Route: {candidates[best_idx].route}")
            print(f"  Adjusted ETA: {candidates[best_idx].features['adj_eta_hours']:.2f} hours")
            print(f"  Capacity utilization: {candidates[best_idx].features['capacity_utilization']:.2%}")

        return optimal_routes

    def save_model(self, path: str):
        """Save trained model"""
        torch.save({
            'model_state_dict': self.model.state_dict(),
        }, path)
        print(f"Model saved to {path}")

    def load_model(self, path: str):
        """Load trained model"""
        checkpoint = torch.load(path, map_location='cpu')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Model loaded from {path}")

# Example usage and testing
def main():
    # Initialize pipeline
    pipeline = FleetOptimizationPipeline()

    # Train model
    print("="*50)
    print("Training Fleet Optimization Model")
    print("="*50)

    train_losses, val_accs = pipeline.train_model(num_scenarios=500, epochs=30)

    # Save model
    pipeline.save_model("fleet_optimization_model.pth")

    # Test on a new scenario
    print("\n" + "="*50)
    print("Testing on New Scenario")
    print("="*50)

    # Create test scenario
    test_scenario = Scenario(
        id="test_scenario",
        vehicles=[
            Vehicle("v1", 300, Location(40.7128, -74.0060), 8, False),
            Vehicle("v2", 250, Location(40.7128, -74.0060), 8, False),
        ],
        orders=[
            Order("o1", Location(40.72, -74.01), 50, (8, 12)),
            Order("o2", Location(40.73, -74.02), 30, (9, 14)),
            Order("o3", Location(40.71, -74.00), 40, (10, 15)),
            Order("o4", Location(40.70, -73.99), 60, (11, 16)),
            Order("o5", Location(40.74, -74.03), 35, (8, 13)),
            Order("o6", Location(40.69, -73.98), 45, (9, 14)),
        ],
        weather_code=1,  # Rain
        hour_of_day=9,
        traffic_index=0.6
    )

    # Optimize routes
    optimal_routes = pipeline.optimize_routes(test_scenario, k_candidates=5)

    print("\n" + "="*50)
    print("Optimization Complete!")
    print("="*50)

    for vehicle_id, route in optimal_routes.items():
        print(f"\nVehicle {vehicle_id} optimal route: {route}")

if __name__ == "__main__":
    main()

Training Fleet Optimization Model
Generating training data...
Training model...
Epoch 10/30 - Loss: 1.3401, Val Acc: 0.0000
Epoch 20/30 - Loss: 1.1171, Val Acc: 0.0000
Epoch 30/30 - Loss: 1.0407, Val Acc: 0.0000
Final validation accuracy: 0.0000
Model saved to fleet_optimization_model.pth

Testing on New Scenario
Vehicle v1: Selected candidate 0 with score 0.305
  Route: ['o3', 'o1', 'o2', 'o5', 'o4']...
  Adjusted ETA: 0.59 hours
  Capacity utilization: 86.67%
Vehicle v2: Selected candidate 0 with score 0.309
  Route: ['o3', 'o1', 'o2', 'o5', 'o4']
  Adjusted ETA: 0.48 hours
  Capacity utilization: 86.00%

Optimization Complete!

Vehicle v1 optimal route: ['o3', 'o1', 'o2', 'o5', 'o4', 'o6']

Vehicle v2 optimal route: ['o3', 'o1', 'o2', 'o5', 'o4']


In [2]:
pipeline = FleetOptimizationPipeline()
train_losses, val_accs = pipeline.train_model(
    num_scenarios=1000,
    epochs=50
)
pipeline.save_model("fleet_optimization_model.pth")

Generating training data...
Training model...
Epoch 10/50 - Loss: 1.0829, Val Acc: 0.0000
Epoch 20/50 - Loss: 0.9718, Val Acc: 0.0000
Epoch 30/50 - Loss: 0.9222, Val Acc: 0.0000
Epoch 40/50 - Loss: 0.8945, Val Acc: 0.0000
Epoch 50/50 - Loss: 0.8597, Val Acc: 0.0000
Final validation accuracy: 0.0000
Model saved to fleet_optimization_model.pth
